In [2]:
import pandas as pd
import torch
import torchaudio
from pathlib import Path
import transformers
device = torch.device("cuda")

C:\Users\akondep1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoFeatureExtractor, WhisperForAudioClassification
import os

token = os.getenv('HF_TOKEN')
model_id = "openai/whisper-tiny"

feature_extractor = AutoFeatureExtractor.from_pretrained(model_id, token=token)
model = WhisperForAudioClassification.from_pretrained(model_id, token=token, num_labels=7)

feature_extractor
model.to(device)

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at openai/whisper-tiny and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'WhisperFeatureExtractor' object has no attribute 'to'

In [ ]:
data_path = ""
df = pd.read_csv("data/ground_truth.csv")

In [ ]:
import os
from functools import lru_cache
from subprocess import CalledProcessError, run
from typing import Optional, Union

import numpy as np
import torch
import torch.nn.functional as F

def exact_div(x, y):
    assert x % y == 0
    return x // y


class AudioUtil():
  # hard-coded audio hyperparameters
  SAMPLE_RATE = 16000
  N_FFT = 400
  HOP_LENGTH = 160
  CHUNK_LENGTH = 30
  N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000 samples in a 30-second chunk
  N_FRAMES = exact_div(N_SAMPLES, HOP_LENGTH)  # 3000 frames in a mel spectrogram input

  N_SAMPLES_PER_TOKEN = HOP_LENGTH * 2  # the initial convolutions has stride 2
  FRAMES_PER_SECOND = exact_div(SAMPLE_RATE, HOP_LENGTH)  # 10ms per audio frame
  TOKENS_PER_SECOND = exact_div(SAMPLE_RATE, N_SAMPLES_PER_TOKEN)  # 20ms per audio token


  def load_audio(file: str, sr: int = SAMPLE_RATE):
      """
      Open an audio file and read as mono waveform, resampling as necessary

      Parameters
      ----------
      file: str
          The audio file to open

      sr: int
          The sample rate to resample the audio if necessary

      Returns
      -------
      A NumPy array containing the audio waveform, in float32 dtype.
      """

      # This launches a subprocess to decode audio while down-mixing
      # and resampling as necessary.  Requires the ffmpeg CLI in PATH.
      # fmt: off
      cmd = [
          "ffmpeg",
          "-nostdin",
          "-threads", "0",
          "-i", file,
          "-f", "s16le",
          "-ac", "1",
          "-acodec", "pcm_s16le",
          "-ar", str(sr),
          "-"
      ]
      # fmt: on
      try:
          out = run(cmd, capture_output=True, check=True).stdout
      except CalledProcessError as e:
          raise RuntimeError(f"Failed to load audio: {e.stderr.decode()}") from e

      return np.frombuffer(out, np.int16).flatten().astype(np.float32) / 32768.0


  def pad_or_trim(array, length: int = N_SAMPLES, *, axis: int = -1):
      """
      Pad or trim the audio array to N_SAMPLES, as expected by the encoder.
      """
      if torch.is_tensor(array):
          if array.shape[axis] > length:
              array = array.index_select(
                  dim=axis, index=torch.arange(length, device=array.device)
              )

          if array.shape[axis] < length:
              pad_widths = [(0, 0)] * array.ndim
              pad_widths[axis] = (0, length - array.shape[axis])
              array = F.pad(array, [pad for sizes in pad_widths[::-1] for pad in sizes])
      else:
          if array.shape[axis] > length:
              array = array.take(indices=range(length), axis=axis)

          if array.shape[axis] < length:
              pad_widths = [(0, 0)] * array.ndim
              pad_widths[axis] = (0, length - array.shape[axis])
              array = np.pad(array, pad_widths)

      return array
  
  @lru_cache(maxsize=None)
  def mel_filters(device, n_mels: int) -> torch.Tensor:
        """
        load the mel filterbank matrix for projecting STFT into a Mel spectrogram.
        Allows decoupling librosa dependency; saved using:

            np.savez_compressed(
                "mel_filters.npz",
                mel_80=librosa.filters.mel(sr=16000, n_fft=400, n_mels=80),
                mel_128=librosa.filters.mel(sr=16000, n_fft=400, n_mels=128),
            )
        """
        assert n_mels in {80, 128}, f"Unsupported n_mels: {n_mels}"

        with np.load("data/mel_filters.npz", allow_pickle=False) as f:
            return torch.from_numpy(f[f"mel_{n_mels}"]).to(device)

  def log_mel_spectrogram(
        audio: Union[str, np.ndarray, torch.Tensor],
        n_mels: int = 80,
        padding: int = 0,
        device: Optional[Union[str, torch.device]] = None,
    ):
        if not torch.is_tensor(audio):
            if isinstance(audio, str):
                audio = AudioUtil.load_audio(audio)
            audio = torch.from_numpy(audio)

        if device is not None:
            audio = audio.to(device)
        if padding > 0:
            audio = F.pad(audio, (0, padding))
        window = torch.hann_window(AudioUtil.N_FFT).to(audio.device)
        stft = torch.stft(audio, AudioUtil.N_FFT, AudioUtil.HOP_LENGTH, window=window, return_complex=True)
        magnitudes = stft[..., :-1].abs() ** 2

        filters = AudioUtil.mel_filters(audio.device, n_mels)
        mel_spec = filters @ magnitudes

        log_spec = torch.clamp(mel_spec, min=1e-10).log10()
        log_spec = torch.maximum(log_spec, log_spec.max() - 8.0)
        log_spec = (log_spec + 4.0) / 4.0
        return log_spec

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torchaudio

# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
            
  # ----------------------------
  # Number of items in dataset
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
  def __getitem__(self, idx):
    # Absolute file path of the audio file - concatenate the audio directory with
    # the relative path
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    # Get the Class ID
    class_id = self.df.loc[idx, 'classID']

    audio = AudioUtil.load_audio(audio_file)
    audio = AudioUtil.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = AudioUtil.log_mel_spectrogram(audio).to(model.device)

    return {
        "input_features": mel,  # Adjust this key according to your model's input name
        "labels": torch.tensor(class_id)
    }

In [ ]:
from torch.utils.data import random_split

myds = SoundDS(df, data_path)

# Split the dataset into training, validation, and test sets
num_items = len(myds)
num_train = round(num_items * 0.7)
num_val = round(num_items * 0.15)
num_test = num_items - num_train - num_val
train_ds, val_ds, test_ds = random_split(myds, [num_train, num_val, num_test])


In [ ]:
from transformers import TrainingArguments, Trainer

batch_size = 32

args = TrainingArguments(
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    output_dir="./"
)

C:\Users\akondep1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

  0%|          | 0/130 [00:00<?, ?it/s]C:\Users\akondep1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\models\whisper\modeling_whisper.py:691: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
                                                
  8%|▊         | 10/130 [01:51<21:51, 10.93s/it]

{'loss': 1.9039, 'grad_norm': 4.723896503448486, 'learning_rate': 2.307692307692308e-05, 'epoch': 0.38}


                                                
 15%|█▌        | 20/130 [03:29<17:35,  9.60s/it]

{'loss': 1.4397, 'grad_norm': 3.453059434890747, 'learning_rate': 2.8205128205128207e-05, 'epoch': 0.77}


 20%|██        | 26/130 [04:19<13:24,  7.74s/it]























AttributeError: module 'datasets.metric' has no attribute 'compute'

In [ ]:
trainer.evaluate()